In [4]:
import matplotlib.pyplot as plt
import math
import numpy as np
import random
import pandas as pd
from scipy import stats

with open('Москва_2021.txt', 'r') as file:
    ages = list(map(int, file.readlines()))

In [8]:
# Задание 1. По критерию Пирсона и уровне значимости 0,05 проверить нулевую гипотезу о нормальном распределении
#-------------------------------------------------#
#Разбиение данных на 7 равных групп
num_bins = 7
bins = np.linspace(start=int(min(ages)), stop=int(max(ages)), num=num_bins + 1, dtype=int)
print(bins)

# Вычисление эмпирическх частот
empirical_freq, _ = np.histogram(ages, bins=bins)
print(empirical_freq)

#Данные нормального распределения
mu=35.37
sigma=144.91

# Вычисление теоретических частот
# Для каждого интервала

theoretical_freq = []

avg_bins = []
for i in range(0, len(bins)-2):
  avg_bins.append((bins[i]+bins[i+1])/2)

theoretical_mu=0
for i in range(len(avg_bins)-1):
  theoretical_mu += avg_bins[i] * empirical_freq[i] / len(ages)
print(theoretical_mu)

sum1 = 0
for x, n in zip(avg_bins, empirical_freq):
  sum1 += n * (x - theoretical_mu)**2
theoretical_sigma = math.sqrt(sum1 / len(ages))
print(theoretical_sigma)

norm_bins = []
for i in range(len(bins)-1):
  norm_bins.append(bins[i] - theoretical_mu)
norm_bins[0] = -math.inf
norm_bins[-1] = math.inf
print(norm_bins)

fi = []
for i in range(len(norm_bins)):
  if (norm_bins[i] == -math.inf):
    fi.append(-0.5)
  elif (norm_bins[i] == math.inf):
    fi.append(0.5)
  elif norm_bins[i] < 0:
    fi.append(-(stats.norm.cdf(norm_bins[i]) - 0.5))
  else:
     fi.append(stats.norm.cdf(norm_bins[i]) - 0.5)
print(fi)


for i in range(len(bins) - 1):
    p = stats.norm.cdf(bins[i + 1], mu, sigma) - stats.norm.cdf(bins[i], mu, sigma)
    theoretical_freq.append(p * len(ages))

# Преобразуем в массив
theoretical_freq = np.array(theoretical_freq)

# Вычисление хи-квадрат
chi2_stat = 0
for i in range(num_bins-1):
  chi2_stat += ( ( empirical_freq[i] - theoretical_freq[i] )**2 ) / ( theoretical_freq[i] )

# Сравнение с критическим значением
alpha = 0.05
critical_value = stats.chi2.ppf(alpha, df=num_bins - 1)

# Вывод результатов
print(f"Chi-squared Statistic: {chi2_stat}")
print(f"Критическое значение: {critical_value}")

if chi2_stat > critical_value:
    print("Отвергаем нулевую гипотезу: данные не распределены нормально.")
else:
    print("Не отвергаем нулевую гипотезу: данные распределены нормально.")

[14 22 30 39 47 56 64 73]
[3867 8571 7674 6452 3583 1362  914]
31.433411467168362
11.70269673443666
[-inf, np.float64(-9.433411467168362), np.float64(-1.4334114671683622), np.float64(7.566588532831638), np.float64(15.566588532831638), np.float64(24.566588532831638), inf]
[-0.5, np.float64(0.5), np.float64(0.4241298583824062), np.float64(0.4999999999999808), np.float64(0.5), np.float64(0.5), 0.5]
Chi-squared Statistic: 216039.22164565255
Критическое значение: 1.6353828943279067
Отвергаем нулевую гипотезу: данные не распределены нормально.
